In [97]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
mdir = Path(Path(os.path.abspath('')).parent)
sys.path.append(str(mdir))
from copy import deepcopy
from copy import copy

import seaborn as sns

In [75]:
pd.set_option('display.max_columns', None)

In [12]:
mdir/'data'/'train.csv'

PosixPath('/Users/denis.semenov/Desktop/Ra1fHak_DS/data/train.csv')

In [13]:
 train = pd.read_csv(mdir/'data'/'train.csv')

/Users/denis.semenov/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
train['city'].unique()

array(['Пермь', 'Шатура', 'Ярославль', ..., 'Липовицы', 'Черемушки',
       'Берсеневка'], dtype=object)

In [20]:
test = pd.read_csv(mdir/'data'/'test.csv')

In [23]:
set(test.city) - set(train.city)

{'Ромашково'}

In [29]:
train[['floor','city']]

,floor,city
0,NaN,Пермь
1,NaN,Шатура
2,NaN,Ярославль
3,NaN,Новокузнецк
4,NaN,Москва
...,...,...
279787,1.0,Томск
279788,1.0,Санкт-Петербург
279789,3.0,Калининград
279790,1.0,Кемерово


In [36]:
train.columns

Index(['city', 'floor', 'id', 'lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_name', 'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in

In [76]:
train.sample(10)

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
35284,Липецк,NaN,COL_36101,52.643050,39.675854,0,1,2,2,0,0,0,0,0,0,0,0,7.033078,Липецк,510152.0,0.470072,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,358.859732,7.515089,0,0,0,0.462905,3,3,5,66920.353982,12,7,265.0,227.0,8.181818,9.714286,1997.666667,2008.857143,Липецкая область,56.50000,S8577,2020-02-02,10,0
251638,Санкт-Петербург,NaN,COL_257032,60.047802,30.344282,0,33,79,185,0,0,0,5,0,8,15,29,12.231786,Санкт-Петербург,5381736.0,0.078756,1,17,29,50,0,0,1,2,0,2,4,12,2,5,14,0,0,0,0,0,0,1,3,8,1,2,4,8,0,20,53,131,0.649688,3.687980,0,0,0,0.317957,14,23,49,28350.655342,62,15,2288.0,817.0,12.357143,14.250000,1983.523810,1986.083333,Санкт-Петербург,7370.20000,S8198,2020-08-09,10,0
67729,Димитровград,NaN,COL_69322,54.220207,49.641342,0,3,3,3,0,0,0,0,0,0,0,0,2.439457,Димитровград,116055.0,0.293035,0,4,8,10,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,73.109615,2.688318,0,0,0,0.712838,0,1,1,13430.000000,34,17,91.0,51.0,1.774194,1.866667,1958.967742,1962.866667,Ульяновская область,200.00000,S6089,2020-02-23,10,0
245829,Коломна,NaN,COL_251108,55.083680,38.793350,6,33,73,126,0,0,1,1,0,1,5,10,2.024046,Коломна,144253.0,0.309587,0,6,14,41,0,0,0,1,0,1,3,5,3,4,6,1,2,2,0,0,1,0,2,4,0,4,9,16,6,26,58,95,87.953691,0.734099,0,0,0,0.310544,8,10,21,55968.750000,147,50,1775.0,780.0,4.062069,5.510204,1957.165517,1971.612245,Московская область,160.00000,S6089,2020-08-02,10,0
239918,"Тальменский район, Тальменка рп",1,COL_245071,53.817223,83.582146,0,0,0,0,0,0,0,0,0,0,0,0,53.841744,Барнаул,632723.0,3.332587,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,138.034545,1.212806,0,0,0,1.756128,0,0,0,3111.806099,9,4,0.0,0.0,NaN,NaN,NaN,NaN,Алтайский край,255.80000,S13596,2020-07-26,10,0
271353,Москва,NaN,COL_277174,55.744183,37.639317,4,53,135,366,0,0,0,4,0,13,34,100,1.533572,Москва,12630289.0,0.186638,0,32,70,158,1,4,8,14,0,8,11,24,4,13,30,4,11,19,1,8,24,2,9,24,0,4,9,26,2,12,41,141,0.694698,1.682271,0,0,0,0.070464,10,23,39,507394.186639,108,16,2464.0,362.0,6.546296,7.000000,1940.731481,1955.937500

In [72]:
city = train[['city','id']].groupby(by='city').id.count().reset_index()
city.sort_values('id', ascending=False)

,city,id
2465,Москва,55996
3416,Санкт-Петербург,18906
1881,Краснодар,13009
1131,Екатеринбург,8580
2726,Новосибирск,6753
...,...,...
2452,"Мокшанский район, пос. Волынский",1
2453,Молоково,1
2456,"Молчановский район, с. Молчаново",1
2466,Московская ул.,1


In [73]:
city = test[['city','id']].groupby(by='city').id.count().reset_index()
city.sort_values('id', ascending=False)

,city,id
57,Красноярск,405
33,Иркутск,228
110,Санкт-Петербург,205
139,Томск,138
99,Петрозаводск,136
...,...,...
69,Маркова,1
66,Луга,1
64,Ленинск-Кузнецкий,1
60,Кузьмоловский,1


In [82]:
train[train['osm_city_nearest_name'].isin(['饶河县', '绥芬河市'])].sample(5) # osm_city_closest_dist туда же - поменять

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
224003,Лучегорск,NaN,COL_228873,46.445065,134.29660,0,12,35,57,0,0,0,0,0,0,0,0,45.045453,饶河县,NaN,0.287445,0,9,20,22,0,0,0,1,0,1,6,8,0,0,0,0,1,3,0,1,2,0,0,0,0,1,1,1,0,10,22,37,2227.329373,8.889401,0,0,0,0.289801,2,5,5,5980.198020,69,35,1684.0,824.0,4.947368,5.107143,1984.711864,1988.142857,Приморский край,101.0,S20214,2020-07-19,10,0
59172,"Пожарский район, с. Светлогорье, Первый мкр",1,COL_60569,46.857033,134.44760,0,1,1,1,0,0,0,0,0,0,0,0,34.086624,饶河县,NaN,14.444392,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,2215.430101,15.932745,0,0,0,46.633121,0,0,0,17500.000000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Приморский край,54.0,S519,2020-02-16,110,0
240050,"Кировский район, Кировский пгт",1,COL_245204,45.088050,133.47316,0,0,0,0,0,0,0,0,0,0,0,0,194.667853,饶河县,NaN,1.757790,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2250.074415,13.750862,0,0,0,6.589910,0,0,0,6439.227731,1,0,2.0,NaN,2.000000,NaN,1987.000000,NaN,Приморский край,455.8,S27843,2020-07-26,10,0
265711,Лесозаводск,1,COL_271430,45.455870,133.38249,0,4,6,8,0,0,0,0,0,1,1,2,156.902488,饶河县,NaN,0.286354,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,4,2222.266362,0.331770,1,1,1,1.396197,0,0,0,17735.042735,3,2,10.0,4.0,2.000000,2.000000,1976.500000,1960.000000,Приморский край,70.2,S19528,2020-08-23,110,0
249322,Лесозаводск,NaN,COL_254681,45.464455,133.39932,0,13,25,29,0,0,0,0,0,3,3,3,155.596889,饶河县,NaN,0.123227,0,6,9,14,0,0,0,1,0,1,3,3,1,4,4,0,1,1,0,1,1,0,0,0,0,0,2,2,0,8,13,15,2222.879704,1.946566,0,0,0,0.113986,4,5,7,61230.769231,70,33,1835.0,1040.0,4.955882,4.818182,1977.867647,1978.969697,Приморский край,78.0,S9396,2020-08-02,110,0


In [86]:
train.date.min(), train.date.max()

('2020-01-05', '2020-08-23')

In [87]:
test.date.min(), test.date.max()

('2020-09-06', '2020-12-27')

In [89]:
train.price_type.unique(), test.price_type.unique()

(array([0, 1]), array([1]))

In [108]:
def get_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Формирует time-признаки
    :param df: dataframe, обучающая выборка
    :return: dataframe
    """
    df_new = train.copy()
    
    df_new['date'] = pd.to_datetime(df_new['date'], format='%Y-%m-%d')

    df_new['year'] = df_new['date'].dt.year 
    df_new['month'] = df_new['date'].dt.month 
    df_new['day'] = df_new['date'].dt.day

    df_new['dayofweek_num'] = df_new['date'].dt.dayofweek  
    df_new['quarter'] = df_new['date'].dt.quarter

    df_new['dayofyear'] = df_new['date'].dt.dayofyear  
    df_new['weekofyear'] = df_new['date'].dt.weekofyear
    
    return df_new

In [109]:
get_time_features(train)

/Users/denis.semenov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,osm_building_points_in_0.01,osm_catering_points_in_0.001,osm_catering_points_in_0.005,osm_catering_points_in_0.0075,osm_catering_points_in_0.01,osm_city_closest_dist,osm_city_nearest_name,osm_city_nearest_population,osm_crossing_closest_dist,osm_crossing_points_in_0.001,osm_crossing_points_in_0.005,osm_crossing_points_in_0.0075,osm_crossing_points_in_0.01,osm_culture_points_in_0.001,osm_culture_points_in_0.005,osm_culture_points_in_0.0075,osm_culture_points_in_0.01,osm_finance_points_in_0.001,osm_finance_points_in_0.005,osm_finance_points_in_0.0075,osm_finance_points_in_0.01,osm_healthcare_points_in_0.005,osm_healthcare_points_in_0.0075,osm_healthcare_points_in_0.01,osm_historic_points_in_0.005,osm_historic_points_in_0.0075,osm_historic_points_in_0.01,osm_hotels_points_in_0.005,osm_hotels_points_in_0.0075,osm_hotels_points_in_0.01,osm_leisure_points_in_0.005,osm_leisure_points_in_0.0075,osm_leisure_points_in_0.01,osm_offices_points_in_0.001,osm_offices_points_in_0.005,osm_offices_points_in_0.0075,osm_offices_points_in_0.01,osm_shops_points_in_0.001,osm_shops_points_in_0.005,osm_shops_points_in_0.0075,osm_shops_points_in_0.01,osm_subway_closest_dist,osm_train_stop_closest_dist,osm_train_stop_points_in_0.005,osm_train_stop_points_in_0.0075,osm_train_stop_points_in_0.01,osm_transport_stop_closest_dist,osm_transport_stop_points_in_0.005,osm_transport_stop_points_in_0.0075,osm_transport_stop_points_in_0.01,per_square_meter_price,reform_count_of_houses_1000,reform_count_of_houses_500,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type,year,month,day,dayofweek_num,quarter,dayofyear,weekofyear
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,0,0,0,0,2,4,6,3.293470,Пермь,1055397.0,0.027732,3,6,17,34,0,0,1,1,0,0,1,2,2,3,4,0,0,1,0,0,0,0,1,2,0,1,2,4,4,11,20,28,269.024986,3.368385,0,0,0,0.002864,4,13,21,139937.500000,136,49,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.000000,S27289,2020-01-05,10,0,2020,1,5,6,1,5,1
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,0,0,1,0,2,2,6,43.950989,Орехово-Зуево,120184.0,0.089441,0,31,50,57,0,1,2,3,0,0,1,2,1,1,3,2,4,6,2,2,2,1,1,2,0,1,2,3,1,12,20,29,102.455451,1.476600,0,0,0,0.154661,4,10,11,60410.714286,146,37,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.000000,S17052,2020-01-05,10,0,2020,1,5,6,1,5,1
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,0,1,1,0,3,6,11,2.676293,Ярославль,603961.0,0.200995,0,15,29,53,0,1,2,2,0,0,5,9,0,1,3,0,0,0,0,0,1,2,3,6,0,1,6,9,1,16,37,80,243.361937,1.455127,0,0,0,0.118275,9,13,21,45164.761264,105,27,1883.0,573.0,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.400000,S16913,2020-01-05,110,0,2020,1,5,6,1,5,1
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,0,0,1,0,0,1,4,15.618563,Новокузнецк,552105.0,0.861400,0,0,0,5,0,0,0,0,0,0,0,1,0,0,3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,4,11,295.633502,9.400855,0,0,0,1.036523,0,0,3,28805.263158,75,2,1801.0,54.0,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.000000,S10148,2020-01-05,110,0,2020,1,5,6,1,5,1
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,1,1,1,0,8,14,26,9.995325,Химки,232066.0,0.236744,0,14,40,78,0,0,0,0,0,0,3,9,2,7,14,0,2,2,1,1,3,3,4,9,0,8,8,12,0,8,33,89,1.110429,1.235503,0,0,0,0.235032,10,32,62,13222.591362,144,38,3090.0,619.0,7.263889,5.684211,1963.229167,1960.500000,Москва,60.200000,S1338,2020-01-05,10,0,2020,1,5,6,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,